# Практическое задание №3. 
# Задача классификации на Kaggle с помощью логистической регрессии (light)

Нефтегазовая отрасль имеет сложную технологическую цепочку, которая начинается с геологической разведки и заканчивается доставкой нефти и газа потребителям. Одной из важных задач является определение места залежей — на суше или в море — на основе различных параметров.

Ваша задача: разработать алгоритм машинного обучения, который позволит классифицировать место залежей нефти и газа.

In [ ]:
!kaggle competitions download -c classification-of-oil-and-gas
!unzip classification-of-oil-and-gas.zip

In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
train_data = pd.read_csv("train_oil.csv")
test_data = pd.read_csv("oil_test.csv")

## Исследование и анализ датасета

In [3]:
print("Train data head:")
print(train_data.head())

Train data head:
     Field name           Reservoir unit       Country               Region  \
0       ZHIRNOV              MELEKESKIAN        RUSSIA  FORMER SOVIET UNION   
1   LAGOA PARDA  LAGOA PARDA (URUCUTUCA)        BRAZIL        LATIN AMERICA   
2        ABQAIQ                   ARAB D  SAUDI ARABIA          MIDDLE EAST   
3     MURCHISON                    BRENT    UK /NORWAY               EUROPE   
4  WEST PEMBINA   NISKU (PEMBINA L POOL)        CANADA        NORTH AMERICA   

           Basin name        Tectonic regime  Latitude  Longitude  \
0          VOLGA-URAL  COMPRESSION/EVAPORITE   51.0000    44.8042   
1      ESPIRITO SANTO              EXTENSION  -19.6017   -39.8332   
2            THE GULF  COMPRESSION/EVAPORITE   26.0800    49.8100   
3  NORTH SEA NORTHERN              EXTENSION   61.3833     1.7500   
4      WESTERN CANADA            COMPRESSION   53.2287  -115.8008   

   Operator company Onshore/Offshore Hydrocarbon type      Reservoir status  \
0  NIZHNEVOLZH

In [4]:
print("Train data info:")
print(train_data.info())

Train data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Field name                      309 non-null    object 
 1   Reservoir unit                  309 non-null    object 
 2   Country                         282 non-null    object 
 3   Region                          271 non-null    object 
 4   Basin name                      271 non-null    object 
 5   Tectonic regime                 309 non-null    object 
 6   Latitude                        282 non-null    float64
 7   Longitude                       279 non-null    float64
 8   Operator company                309 non-null    object 
 9   Onshore/Offshore                309 non-null    object 
 10  Hydrocarbon type                309 non-null    object 
 11  Reservoir status                309 non-null    object 
 12  Structural setting 

Данные пригодны для задач классификации, регрессии или кластеризации в области нефтегазового сектора. Однако наличие пропущенных значений требует предварительной обработки перед применением моделей машинного обучения.

In [5]:
# разделение признаков на числовые и категориальные:
numeric_features = [
    "Latitude", 
    "Longitude", 
    "Depth", 
    "Thickness (gross average ft)",
    "Thickness (net pay average ft)",
    "Porosity",
    "Permeability"
]
categorical_features = [
    "Reservoir unit",
    "Country",
    "Region",
    "Basin name",
    "Tectonic regime",
    "Operator company",
    "Hydrocarbon type",
    "Reservoir status",
    "Structural setting",
    "Reservoir period",
    "Lithology"
]

## Предобработка данных

In [6]:
# аполнение пропусков в числовых колонках медианой:
train_data[numeric_features] = train_data[numeric_features].fillna(train_data[numeric_features].median(numeric_only=True))

# заполнение пропусков в категориальных колонках модой:
mode_values = train_data[categorical_features].mode().iloc[0]  # Берем первую строку мод
train_data[categorical_features] = train_data[categorical_features].fillna(mode_values)

median_values = train_data[numeric_features].median()
print(f"Median:\n{median_values}")
print(f"\nMode:\n{mode_values}")

Median:
Latitude                            38.135
Longitude                            1.810
Depth                             6500.000
Thickness (gross average ft)       312.000
Thickness (net pay average ft)     110.000
Porosity                            17.000
Permeability                        68.000
dtype: float64

Mode:
Reservoir unit                       BRENT
Country                                USA
Region                       NORTH AMERICA
Basin name                  WESTERN CANADA
Tectonic regime                COMPRESSION
Operator company                  NUMEROUS
Hydrocarbon type                       OIL
Reservoir status      DECLINING PRODUCTION
Structural setting                FORELAND
Reservoir period                CRETACEOUS
Lithology                        SANDSTONE
Name: 0, dtype: object


In [7]:
print("Train data info:")
print(train_data.info())

Train data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Field name                      309 non-null    object 
 1   Reservoir unit                  309 non-null    object 
 2   Country                         309 non-null    object 
 3   Region                          309 non-null    object 
 4   Basin name                      309 non-null    object 
 5   Tectonic regime                 309 non-null    object 
 6   Latitude                        309 non-null    float64
 7   Longitude                       309 non-null    float64
 8   Operator company                309 non-null    object 
 9   Onshore/Offshore                309 non-null    object 
 10  Hydrocarbon type                309 non-null    object 
 11  Reservoir status                309 non-null    object 
 12  Structural setting 

После предобработки в датасете не осталось пропущенных значений.

In [8]:
random_state = 42

In [9]:
# Разделение на признаки и целевую переменную:
X = train_data.drop(columns=["Field name", "Onshore/Offshore"])
y = train_data["Onshore/Offshore"]

# Разделение на обучающую и валидационную выборки:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=random_state, 
    stratify=y
)

In [10]:
# Предобработка признаков:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ])

## Модели

Протестируем 3 модели:

    * Логистическая регрессия;
    * Случайный лес;
    * Градиентный бустинг.

In [11]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Гиперпараметры для каждой модели:
param_grids = {
    "Logistic Regression": {"classifier__C": [0.1, 1, 10],
                            "classifier__solver": ["lbfgs"]},
    "Random Forest": {"classifier__n_estimators": [100, 200],
                      "classifier__max_depth": [None, 10, 20]},
    "Gradient Boosting": {"classifier__learning_rate": [0.1, 0.05],
                          "classifier__n_estimators": [100, 200]}
}

best_models = {}

# Обучение моделей:
for model_name, model in models.items():
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", model)
    ])

    grid_search = GridSearchCV(
        pipeline,
        param_grids[model_name],
        cv=3,
        scoring="accuracy",
        error_score="raise"  # чтобы увидеть реальную ошибку при необходимости
    )
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    best_models[model_name] = best_model

In [12]:
print("Оценка точности моделей\n")
for model_name, model in best_models.items():
    print(f"\nМодель {model_name}:")
    y_pred = model.predict(X_val)
    print(classification_report(y_val, y_pred, zero_division=0))
    print("Accuracy:", accuracy_score(y_val, y_pred))

Оценка точности моделей


Модель Logistic Regression:
                  precision    recall  f1-score   support

        OFFSHORE       0.85      0.65      0.73        17
         ONSHORE       0.86      0.95      0.90        44
ONSHORE-OFFSHORE       0.00      0.00      0.00         1

        accuracy                           0.85        62
       macro avg       0.57      0.53      0.55        62
    weighted avg       0.84      0.85      0.84        62

Accuracy: 0.8548387096774194

Модель Random Forest:
                  precision    recall  f1-score   support

        OFFSHORE       1.00      0.47      0.64        17
         ONSHORE       0.81      1.00      0.90        44
ONSHORE-OFFSHORE       0.00      0.00      0.00         1

        accuracy                           0.84        62
       macro avg       0.60      0.49      0.51        62
    weighted avg       0.85      0.84      0.81        62

Accuracy: 0.8387096774193549

Модель Gradient Boosting:
                  pr

Видна проблема с классом "ONSHORE-OFFSHORE": слишком малое количество записей с данным классом - ни одна модель не может обучиться на таком маленьком классе.

Все модели показывают схожую точность (accuracy), но поведение разное по классам.

Модель "Logistic Regression":

    Плюсы: 
        * хорошее сочетание precision и recall.
    Минусы: 
        * умеренный recall для OFFSHORE.

Модель "Random Forest":

    Плюсы:
        * когда предсказывает "OFFSHORE", всегда правильно;
        * находит все "ONSHORE";
    Минусы:
        * пропускает больше половины "OFFSHORE".

Модель "Gradient Boosting":

    Плюсы: 
        * лучший среди всех моделей F1 (weighted);
        * лучший среди всех моделей F1 OFFSHORE.
    Минусы: 
        * умеренный recall для OFFSHORE.

Лучшая модель: Gradient Boosting

## Финальное предсказание

In [13]:
# предобработка данных:
test_data[numeric_features] = test_data[numeric_features].fillna(median_values)
test_data[categorical_features] = test_data[categorical_features].fillna(mode_values)
    
X_test = test_data.drop(columns=["Field name"])

# предсказание:
final_model = best_models["Gradient Boosting"]
predictions = final_model.predict(X_test)

# Сохранение результата
pd.DataFrame({'Predictions': predictions}).to_csv('prediction_results.csv', index=False)